# Download Images

<a href="http://msvocds.blob.core.windows.net/coco2014/train2014.zip">Training Daters</a>

# VGG

In [3]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Input
from keras.utils.data_utils import get_file
import keras.backend as K
import h5py
import numpy as np

WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

MEAN_PIXEL = np.array([ 123.68, 116.779, 103.939])

def vgg_layers(img_input, input_shape):
    # Block 1
    img_input = Input(tensor=img_input, shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    x = GlobalMaxPooling2D()(x)

    return x


def load_weights(model):
    weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                            WEIGHTS_PATH_NO_TOP,
                            cache_subdir='models',
                            file_hash='253f8cb515780f3b799900260a226db6')
    f = h5py.File(weights_path)
    layer_names = [name for name in f.attrs['layer_names']]

    for layer in model.layers:
        if layer.name in layer_names:
            g = f[layer.name]
            weights = [g[name] for name in g.attrs['weight_names']]
            layer.set_weights(weights)
            layer.trainable = False

    return model


def VGG19(img_input, input_shape):
    """
    VGG19, but can take input_tensor, and load weights on VGG layers only
    """
    model = Model(img_input, vgg_layers(img_input, input_shape), name='vgg19')
    model = load_weights(model)
    return model


def preprocess_input(x):
    return x - MEAN_PIXEL


# Loss Function

In [4]:
from functools import reduce

from keras.models import Model, Sequential
from keras.layers import Input
from keras import backend as K
import numpy as np
import tensorflow as tf

STYLE_LAYERS = ('block1_conv1', 'block2_conv1',
                'block3_conv1', 'block4_conv1',
                'block5_conv1')

CONTENT_LAYER = 'block4_conv2'

CONTENT_TRAINING_SIZE = (256, 256, 3)

# TODO: remove need for this
BATCH_SIZE = 1
BATCH_SHAPE = (BATCH_SIZE, 256, 256, 3)

def tensor_size(x):
    return np.nanprod(np.array(K.int_shape(x), dtype=np.float))

def l2_loss(x):
    return K.sum(K.square(x)) / 2

def get_vgg_features(input, layers, input_shape):
    if len(K.int_shape(input)) == 3:
        input = K.expand_dims(input, axis=0)
    input = preprocess_input(input)
    vgg = VGG19(input, input_shape)
    outputs = [layer.output for layer in vgg.layers if layer.name in layers]
    return outputs
  

def calculate_content_loss(content_image, reconstructed_image,
                           content_weight, image_shape):
    content_features = get_vgg_features(
            content_image, CONTENT_LAYER, image_shape)[0]
    reconstructed_content_features = get_vgg_features(
            reconstructed_image, CONTENT_LAYER, image_shape)[0]
   
    content_size = tensor_size(content_features)
    content_loss = content_weight * (2 * l2_loss(
        reconstructed_content_features - content_features) / content_size)
    
    return content_loss
    
def calculate_style_loss(style_image, reconstructed_image,
                         style_weight, style_image_shape, content_image_shape):
     # Get outputs of style and content images at VGG layers
    style_vgg_features = get_vgg_features(
            style_image, STYLE_LAYERS, style_image_shape)
    reconstructed_style_vgg_features = get_vgg_features(
            reconstructed_image, STYLE_LAYERS, content_image_shape)
    
    # Calculate the style features and content features
    # Style features are the gram matrices of the VGG feature maps
    style_grams = []
    style_rec_grams = []
    for features in style_vgg_features:
        _, h, w, filters = K.int_shape(features)
        # shape in K.reshape needs to be np.array to convert Dimension to int
        # (should be fixed in newer versions of Tensorflow)
        features = K.reshape(features, np.array((1, h * w, filters)))

        features_size = tensor_size(features)
        features_T = tf.transpose(features, perm=[0,2,1])
        gram = tf.matmul(features_T, features) / features_size
        style_grams.append(gram)
        
    for features in reconstructed_style_vgg_features:
        _, h, w, filters = K.int_shape(features)
        
        # None checks
        # TODO: Can someone else look over this - without this check compile fails throwing an error of Nonetype * Nonetype is invalid
        # this stems from h * w which are set to None in the compile step.
        if h == None or w == None:
            continue
        size = h * w
        features = K.reshape(features, np.array((BATCH_SIZE, size, filters)))
        
        features_size = tensor_size(features)
        features_T = tf.transpose(features, perm=[0,2,1])
        gram = tf.matmul(features_T, features) / features_size
        style_rec_grams.append(gram)       
        
    # Style loss
    style_losses = []
    for style_gram, style_rec_gram in zip(style_grams, style_rec_grams):
        style_gram_size = tensor_size(style_gram)
        l2 = l2_loss(style_gram - style_rec_gram)
        style_losses.append(2 * l2 / style_gram_size)
    
    style_loss = style_weight * reduce(tf.add, style_losses, 0) / BATCH_SIZE
    
    return style_loss
    
    
def calculate_tv_loss(x, tv_weight):
    tv_y_size = tensor_size(x[:,1:,:,:])
    tv_x_size = tensor_size(x[:,:,1:,:])
    y_tv = l2_loss(x[:,1:,:,:] - x[:,:BATCH_SHAPE[1]-1,:,:])
    x_tv = l2_loss(x[:,:,1:,:] - x[:,:,:BATCH_SHAPE[2]-1,:])
    tv_loss = tv_weight*2*(x_tv/tv_x_size + y_tv/tv_y_size)/BATCH_SIZE
    return tv_loss


def create_loss_fn(style_image, content_weight, style_weight, tv_weight):
    style_image = tf.convert_to_tensor(style_image)

    def style_transfer_loss(y_true, y_pred):
        """
        y_true - content_image
        y_pred - reconstructed image
        """
        content_image = y_true
        reconstructed_image = y_pred
        
        content_loss = calculate_content_loss(content_image,
                reconstructed_image, content_weight, CONTENT_TRAINING_SIZE)
        style_loss = calculate_style_loss(style_image,
                reconstructed_image, style_weight, K.int_shape(style_image),
                CONTENT_TRAINING_SIZE)
        tv_loss = calculate_tv_loss(reconstructed_image, tv_weight)

        loss = content_loss + style_loss + tv_loss
        return loss

    return style_transfer_loss


# Transform

In [5]:
from keras.layers import (Conv2D, Conv2DTranspose,
        BatchNormalization, Input, Lambda)
from keras.models import Model
import keras.layers

def Conv2DBatchNorm(inputs, filters, kernel_size,
                    strides=1, activation='relu'):
    return BatchNormalization()(
            Conv2D(
                filters,
                (kernel_size, kernel_size),
                strides=strides,
                activation=activation,
                padding='same'
            )(inputs))

def Conv2DTransposeBatchNorm(inputs, filters, kernel_size,
                              strides=1, activation=None):
    return BatchNormalization()(
            Conv2DTranspose(
                filters,
                (kernel_size, kernel_size),
                strides=strides,
                activation=activation,
                padding='same'
            )(inputs))

def Conv2DResidualBlock(inputs):
    tmp     = Conv2DBatchNorm(inputs, 128, 3)
    tmp2    = Conv2DBatchNorm(tmp, 128, 3, activation=None)
    return keras.layers.add([tmp, tmp2]) 

# TODO: instance norm, init weights?
def TransformNet(inputs):
    conv1   = Conv2DBatchNorm(inputs, 32, 9)
    conv2   = Conv2DBatchNorm(conv1, 64, 3, strides=2)
    conv3   = Conv2DBatchNorm(conv2, 128, 3, strides=2)
    resid1  = Conv2DResidualBlock(conv3)
    resid2  = Conv2DResidualBlock(resid1)
    resid3  = Conv2DResidualBlock(resid2)
    resid4  = Conv2DResidualBlock(resid3)
    resid5  = Conv2DResidualBlock(resid4)
    conv_t1 = Conv2DTransposeBatchNorm(conv3, 64, 3, strides=2)
    conv_t2 = Conv2DTransposeBatchNorm(conv_t1, 32, 3, strides=2)
    conv_t3 = Conv2DBatchNorm(conv_t2, 3, 9, activation='tanh')
    preds = Lambda(lambda x : x * 150 + 255./2)(conv_t3)
    return preds


# Style Transfer

In [15]:
from keras.preprocessing.image import ImageDataGenerator

def create_gen(img_dir, target_size, batch_size):
    datagen = ImageDataGenerator()
    gen = datagen.flow_from_directory(img_dir, target_size=target_size,
                                      batch_size=batch_size, class_mode=None)

    def tuple_gen():
        for img in gen:
            yield (img, img)

    return tuple_gen()


In [ ]:
from keras.models import Model
import keras.backend as K
from keras.layers import Input
from keras.preprocessing import image
from scipy.misc import imresize
import numpy as np

CONTENT_WEIGHT = 0.5
STYLE_WEIGHT = 100
TV_WEIGHT = 200

style_img_path = 'wave.jpg'
style_img = image.load_img(style_img_path)
style_target = image.img_to_array(style_img)

# Needed so that certain layers function in training mode (batch norm)
K.set_learning_phase(1)

inputs = Input(shape=(256, 256, 3))
transform_net = TransformNet(inputs)
model = Model(inputs=inputs, outputs=transform_net)

loss_fn = create_loss_fn(style_target, CONTENT_WEIGHT, STYLE_WEIGHT, TV_WEIGHT)

model.compile(optimizer='adam', loss=loss_fn)

gen = create_gen('data', target_size=(256, 256), batch_size=1)

model.fit_generator(gen, steps_per_epoch=82783)